# HW2: Sistema Criptográfico de chave pública de Rabin

>### Grupo: 
>>André Araújo A87987
\
>>Miguel Gonçalves A90416
\
>>Paulo Costa A87986

## Descrição: 

> Pretende-se que implemente o sistema criptográfico de chave pública de Rabin.  \
\
> Ver, por exemplo, secção 8.3 do Cap 8 do livro Handbook of Applied Cryptography, A. Menezes, P. van Oorschot, S. Vanstone, CRC Press, 1996. (ver na área "conteúdo"). \
\
> Algumas observações: a mensagem a ser cifrada terá que ser, de alguma forma, codificada (isto porque a decifração produz 4 alternativas). Uma forma simples é transformar a mensagem original na sua repetição (ou usar alguns dígitos e repeti-los). Por exemplo, a mensagem 1234 é transformada no texto limpo 12341234.
Pode, também, usar a função sqrt do sagemath para calcular a raiz quadrada modular, mod um primo. Nesse caso, deverá definir o corpo Zp e considerar o elemento nessa estrutura.  Depois, basta usar o Teorema Chinês dos Restos. \
> Deverá: 
>>- descrever o sistema de Rabin; 
>>- exemplificar; 
>>- implementar as funções de criação de chave, cifração e decifração; 
>>- discutir a premissa de segurança. 

## Introdução:

#### 8.10 Gerar chave para a encriptação da public-key de Rabin

Sumário: Cada entidade cria uma chave pública e uma chave privada correspondente.
Cada entidade A deve fazer o seguinte:
1. Gerar dois números primos grandes aleatórios (e distintos) p e q, com aproximadamente o mesmo tamanho.
2. Calcular n = pq.
3. A chave pública de A é n e a chave privada é (p,q).

#### 8.11 Encriptação da public-key de Rabin
Sumário: B vai encriptar uma mensagem m para A, no qual A vai decifrar.
1. Encriptação. B deve fazer o seguinte:
>**(a)** Obter a chave pública autêntica n de A.\
>**(b)** Representar a mensagem como um inteiro m no intervalo {0, 1,... ,n − 1}.\
>**(c)** Calcular $c = m^2 mod n.$\
>**(d)** Enviar o texto encriptado c para A.
2. Decifrar. Para recuperar o texto simples m de c, A deve fazer o seguinte:
>**(a)** Usar o algoritmo 3.44 para encontrar as quatro raízes quadradas m1, m2, m3 e m4 de c modulo n (Ver também a nota 8.12).\
>**(b)** A mensagem enviada foi m1,m2,m3 ou m4. A nota 8.14 decide qual destes é m.

### Exemplo do sistema criptográfico de chave pública de Rabin

>Suponhamos que o Paulo pretende enviar uma mensagem x ao André (pode ser, por exemplo, o pin do cartão MB). O André cria uma chave pública e uma chave privada. O Paulo consulta a chave pública de André e cifra a mensagem, enviando o criptograma y ao André. O André, usando a chave privada, decifra y e obtém a mensagem original x. \
\
>Uma frase que acaba por exemplificar bastante bem, os passos que já a seguir na implementação das funções iremos ter em conta.

## Resolução:

In [15]:
# função auxiliar que duplica a mensagem #
def duplica(i):
    mens = ZZ(i)
    mm = str(mens)
    mmmm = mm+mm
    #print (mmmm)
    str_to_int = int(mmmm)
    return str_to_int

# função auxiliar que recebe uma lista de possiveis mensagens, encontra a mensagem com elementos repetidos e
# retira os duplicados. Retornando essa mensagem pois é a a mensagem correta!
def singl(ms):
    ms=[str(m) for m in ms]
    for c in ms:
        if (c[0:len(c)//2]==c[len(c)//2:]):
            m=c[0:len(c)//2]
            return int(m)

In [16]:
# função que gera as chaves privadas e a chave publica
def generate():
    nbits=30
    m=2^nbits
    M=2^(nbits-1)
    p=random_prime(m, lbound=M)
    while(mod(p,4)!=3):
        p=random_prime(m, lbound=M)
    q=random_prime(m, lbound=M)
    while(p==q or mod(q,4)!=3):
        q=random_prime(m, lbound=M)
    n=p*q
    return (n,(p,q))

In [17]:
# função que recebe uma mensagem e a chave publica, duplica a mensagem e encripta a mensagem
def encryption(m,pubKey):
    n=pubKey
    m1=duplica(m)
    if m1>=n:
        print('mensagem muito grande tente com outra mais pequena')
        return
    c=power_mod(m1, 2, n)
    return c

### Nota 8.12
>(*Encontrar raízes quadradas de c modulo n = pq quando p ≡ q ≡ 3 (mod 4)*) Se p e q forem ambos escolhido como ≡ 3 (mod 4), então o algoritmo 3.44 calcula as quatro raízes quadradas de c modulo n da seguinte forma: \
\
>**1.** Usar o algoritmo de Euclides estendido (2.107) para encontrar inteiros a e b que satisfaçam ap + bq = 1. Observe que a e b podem ser calculados de uma vez só enquanto estamos a gerar a chave (8.10). \
>**2.** Calcular $r = c^{(p+1)/4} mod p$. \
>**3.** Calcular $s = c^{(q+1)/4} mod q$. \
>**4.** Calcular x = (aps + bqr) mod n. \
>**5.** Calcular y = (aps − bqr) mod n. \
>**6.** As quatro raízes quadradas de c modulo n vão ser x, −x mod n, y, e −y mod n.

In [18]:
# função que recebe uma mensagem e as chaves privadas e publica e decripta a mensagem
def decryption(c,key):
    pubKey,privKey=key
    n=pubKey
    p,q=privKey
    r,a,b=xgcd(p,q)
    if r!=1:
        print('erro')
        return
    r=power_mod(c,(p+1)//4,p)
    s=power_mod(c,(q+1)//4,q)
    k=a*p*s
    l=b*q*r
    x=mod(k+l,n)
    y=mod(k-l,n)
    ms=[x,mod(-x,n),y,mod(-y,n)]
    print(ms)
    return singl(ms)
    

In [19]:
key=generate()
pubKey,privKey=key

In [20]:
c=encryption(456,pubKey)
c

208352079936

In [21]:
decryption(c,key)

[290852694740825157, 275454773058224720, 456456, 566307467798593421]


456

## Discutir a premissa de segurança.

>(i) A tarefa enfrentada por um adversário passivo é recuperar o texto simples m do texto cifrado correspondente c. Lembrando-se de que os problemas de fatorar ne calcular o módulo n de raízes quadradas são computacionalmente equivalentes. Portanto, assumindo que a fatoração n é computacionalmente intratável, o esquema de criptografia de chave pública de Rabin é comprovadamente seguro contra um adversário passivo. \
\
>(ii) Embora comprovadamente seguro contra um adversário passivo, o esquema de criptografia de chave pública Rabin sucumbe a um ataque de texto cifrado escolhido. Esse ataque pode ser montado da seguinte maneira. O adversário seleciona um inteiro aleatório  m∈Z∗n  e calcula  c=m2modn . O adversário então apresenta c para a máquina de descriptografia de A, que descriptografa c e retorna algum texto simples y. Uma vez que A não conhece m, e m é escolhido aleatoriamente, o texto simples y não é necessariamente o mesmo que m. Com probabilidade  1/2 ,  y≠±mmodn , em cujo caso gcd (m - y, n) é um dos fatores primos de n. Se  y≡±mmodn , então o ataque é repetido com um novo m. \
\
>(iii) O esquema de criptografia de chave pública Rabin é suscetível a ataques semelhantes aos do RSA. Como é o caso do RSA, alguns ataques podem ser contornados com o salt da mensagem de texto simples, enquanto outros o ataque pode ser evitado adicionando redundância apropriada antes da criptografia

## Conclusão:

>Em estilo de conclusão, achamos que os objetivos do exercício foram alcançados. Isto, pois conseguimos implementar sistema criptográfico de chave pública de Rabin de uma forma, que a nosso ver foi, muito bem conseguida.
\
\
>Esperemos que os professores também achem o mesmo e assim, damos como concluído o segundo trabalho relativo à UC **Teoria de Números Computacional** e esperemos que continuemos com este sucesso nos próximos!

